### Install the necessary packages

In [ ]:
!add-apt-repository -y ppa:alex-p/tesseract-ocr-devel
!apt-get update
!apt-get install tesseract-ocr
!pip install pytesseract

Repository: 'deb https://ppa.launchpadcontent.net/alex-p/tesseract-ocr-devel/ubuntu/ jammy main'
Description:
This PPA contains an OCR engine - libtesseract and a command line program - tesseract. The development version available here (currntly 5.0.0 ) is better in many aspects (functionality, speed, stability) but is not 100 % API compatible with version 4.0. Tesseract 4 added a new neural net (LSTM) based OCR engine which is focused on line recognition, but also still supports the legacy Tesseract OCR engine of Tesseract 3 which works by recognizing character patterns. Compatibility with Tesseract 3 is enabled by using the Legacy OCR Engine mode (--oem 0). It also needs traineddata files which support the legacy engine, for example those from the tessdata repository. Tesseract has unicode (UTF-8) support, and can recognize more than 100 languages "out of the box". Tesseract supports various output formats: plain text, hOCR (HTML), PDF, invisible-text-only PDF, TSV. The master branch

### Download the code zip file

In [ ]:
# !dpkg --configure -a
# !export PATH=/usr/local/lib/python3.10/dist-packages/pytesseract:$PATH
!wget "https://onedrive.live.com/download?cid=DBD24D3DC9712F7A&resid=DBD24D3DC9712F7A%2110213&authkey=AKAJrobqemGhan8" -O multi_column_data.zip
!unzip -qq multi_column_data.zip
%cd multi_column_data

In [ ]:
!wget https://github.com/doctorai-in/upload-images/raw/main/atevH.jpg

## Blog Post Code

### Import Packages

In [ ]:
# import the necessary packages
!pip install img2table
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt
from pytesseract import Output
from tabulate import tabulate
import pandas as pd
import numpy as np
import pytesseract
import argparse
import imutils
import cv2
import os
from google.colab.patches import cv2_imshow
from imutils import contours
import img2table
from img2table.ocr import TesseractOCR
from img2table.document import Image
from PIL import Image as im

### Function to display images in Jupyter Notebooks and Google Colab

In [ ]:
def plt_imshow(title, image):
	# convert the image frame BGR to RGB color space and display it
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	plt.imshow(image)
	plt.title(title)
	plt.grid(False)
	plt.show()

In [ ]:
def find_red_contours(image):
  """
  Finds contours of red-colored objects in the image.

  Args:
      image: The input image (BGR format).

  Returns:
      A list of contours representing red objects in the image.
  """
  # Convert BGR to HSV color space
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  # Define lower and upper bounds for red color in HSV
  # lower boundary RED color range values; Hue (0 - 10)
  lower1 = np.array([0, 20, 120])
  upper1 = np.array([5, 130, 220])

  # upper boundary RED color range values; Hue (160 - 180)
  lower2 = np.array([335,20,120])
  upper2 = np.array([360,130,220])

  lower_mask = cv2.inRange(hsv, lower1, upper1)
  upper_mask = cv2.inRange(hsv, lower2, upper2)

  mask = cv2.bitwise_or(lower_mask, upper_mask)

  # Find contours in the mask
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Return the list of red contours
  return contours

def cut_out_red_numbers(image):
  """
  Cuts out the regions containing red contours (assumed to be numbers).

  Args:
      image: The input image (BGR format).

  Returns:
      A list of images containing the regions with red contours.
  """
  # Find red contours
  red_contours = find_red_contours(image)

  # Initialize list to store cut-out regions
  cut_images = []

  # Loop through each red contour and extract the corresponding region
  for cnt in red_contours:
    x, y, w, h = cv2.boundingRect(cnt)
    cut_image = image[y:y+h, x:x+w]
    cut_images.append(cut_image)

  return cut_images



In [ ]:
def get_dominant_color(image):
    """Calculates the dominant color of an image using K-Means clustering.

    Returns:
        tuple: A tuple containing the dominant color (B, G, R) or None if clustering fails.
    """
    image_reshaped = image.reshape((image.shape[0] * image.shape[1], image.shape[2])).astype(np.float32)
    num_clusters = 5  # Adjust this parameter if needed

    # Create a TermCriteria object for K-Means termination
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

    try:
        # Apply K-Means clustering
        kmeans = cv2.kmeans(image_reshaped, num_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        dominant_color = kmeans[1][0]  # Access the first centroid (dominant color)
        return dominant_color.astype(np.uint8)  # Convert to int
    except:
        print("K-Means clustering failed for this image.")
        return None  # Return None if clustering fails

def find_most_red_image(images):
    most_red_image = None
    highest_red_score = 0.0

    for image in images:
        try:
            dominant_color = get_dominant_color(image)
            if dominant_color is None:  # Skip if K-Means fails
                continue

            if len(dominant_color) == 3:  # Ensure dominant_color has expected length
                red_score = dominant_color[2] * 0.8 + count * 0.2  # Use K-Means color
            else:
                red_score = np.mean(image[:, :, 2])  # Fallback to image average

            if red_score > highest_red_score:
                highest_red_score = red_score
                most_red_image = image
        except Exception as e:
            print(f"Error processing image: {e}")

    return most_red_image

def preprocess_digit_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply binary thresholding to obtain a binary image
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    digit_images = []
    for cnt in contours:
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(cnt)

        # Crop the digit from the original image
        digit_roi = image[y:y+h, x:x+w]

        # Resize and normalize the digit
        resized = cv2.resize(digit_roi, (28, 28), interpolation=cv2.INTER_AREA)
        normalized = resized.astype('float32') / 255.0

        digit_images.append(normalized)

    return digit_images


### Implementing Multi-Column Table OCR

In [ ]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
# 	help="path to input image to be OCR'd")
# ap.add_argument("-o", "--output", required=True,
# 	help="path to output CSV file")
# ap.add_argument("-c", "--min-conf", type=int, default=0,
# 	help="minimum confidence value to filter weak text detection")
# ap.add_argument("-d", "--dist-thresh", type=float, default=25.0,
# 	help="distance threshold cutoff for clustering")
# ap.add_argument("-s", "--min-size", type=int, default=2,
# 	help="minimum cluster size (i.e., # of entries in column)")
# args = vars(ap.parse_args())

# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values # michael_jordan_stats.png # atevH.jpg
args = {
	"image": "atevH.jpg",
	"output": "results.csv",
	"min_conf": 0,
	"dist_thresh": 25.0,
	"min_size": 2,
}

In [ ]:
#Connecting drive with the collab page
from google.colab import drive
drive.mount('/content/drive')

#Setting the location of the file to the drive file containing the input data.
os.chdir('/content/drive/My Drive/test_file_final_proj')


In [ ]:
count =0
markTables =[]
for filename in os.listdir('/content/drive/My Drive/test_file_final_proj'):
    question_tables = []
    if filename.endswith("jpg"):
        # load the input image and convert it to grayscale
        image = cv2.imread(filename);
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # initialize a rectangular kernel that is ~5x wider than it is tall,
        # then smooth the image using a 3x3 Gaussian blur and then apply a
        # blackhat morpholigical operator to find dark regions on a light
        # background
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (51, 11))
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

        # compute the Scharr gradient of the blackhat image and scale the
        # result into the range [0, 255]
        grad = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        grad = np.absolute(grad)
        (minVal, maxVal) = (np.min(grad), np.max(grad))
        grad = (grad - minVal) / (maxVal - minVal)
        grad = (grad * 255).astype("uint8")

        # apply a closing operation using the rectangular kernel to close
        # gaps in between characters, apply Otsu's thresholding method, and
        # finally a dilation operation to enlarge foreground regions
        grad = cv2.morphologyEx(grad, cv2.MORPH_CLOSE, kernel)
        thresh = cv2.threshold(grad, 0, 255,
        cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        thresh = cv2.dilate(thresh, None, iterations=3)
        # plt_imshow("Thresh", thresh)
        # find contours in the thresholded image and grab the largest one,
        # which we will assume is the stats table
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)

        detected_tables = []
        for i in range(len(cnts)):
            table_cnt = cnts[i]
            (x, y, w, h) = cv2.boundingRect(table_cnt)
            table = image[y:y + h, x:x + w]

            #converting images into binary edge files and saving them to detected_tables
            img = cv2.cvtColor(table, cv2.COLOR_BGR2GRAY)
            assert img is not None, "file could not be read, check with os.path.exists()"
            edges = cv2.Canny(img,100,250)
            #inverting the processed image
            edges = cv2.bitwise_not(edges)

            detected_tables.append(edges)

            # plt_imshow("Table",table)
            if table.size != 0:  # Check if table is not empty
                #Detecting the tables that contains the mark data by using pytesseract ocr by detecting the keyword "Question"
                options = "--psm 3"
                results = pytesseract.image_to_string(
                    cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB),
                    config=options
                )

                if(results.find("Question") != -1):
                  question_tables.append(table)
                  plt_imshow("Table",edges)
                  count+=1

        # print(len(question_tables)) #Prints the number of question tables detected in a answer sheet.

        mostRedImage = find_most_red_image(question_tables)
        if mostRedImage is not None:
          plt_imshow("Red Image", mostRedImage)
          markTables.append(mostRedImage)


        # The below lines contains the code to detect the elements with the most red elements in it but it is not working to find the table that contains the marks.
        # mark_table = find_most_red_image(detected_tables)
        # cv2_imshow(mark_table)
print(count)

## The below code block contains a number of different approaches to detect text regions in a table

In [ ]:
#This method uses open cv libraries to try and remove the table in order to find out the text regions.
def findTextRegion1(image):
    original = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 130, 255, 1)

    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,2))
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,1))
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    erode = cv2.erode(canny, vertical_kernel)
    print("remove horizontal")
    cv2_imshow(erode)
    dilate = cv2.dilate(erode, vertical_kernel, iterations=5)
    print('dilate vertical')
    cv2_imshow(dilate)
    erode = cv2.erode(dilate, horizontal_kernel, iterations=1)
    print('remove vertical')
    cv2_imshow(erode)
    dilate = cv2.dilate(erode, kernel, iterations=4)
    print('dilate horizontal')
    cv2_imshow(dilate)

    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    digit_contours = []
    for c in cnts:
        area = cv2.contourArea(c)
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.01 * peri, True)
        x,y,w,h = cv2.boundingRect(approx)
        aspect_ratio = w / float(h)

        if (aspect_ratio >= 0.4 and aspect_ratio <= 1.3):
            if area > 150:
                ROI = original[y:y+h, x:x+w]
                cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
                digit_contours.append(c)

    sorted_digit_contours = contours.sort_contours(digit_contours, method='left-to-right')[0]
    contour_number = 0
    for c in sorted_digit_contours:
        x,y,w,h = cv2.boundingRect(c)
        ROI = original[y:y+h, x:x+w]
        character = cv2.cvtColor(table, cv2.COLOR_BGR2RGB)
        assert img is not None, "file could not be read, check with os.path.exists()"
        cv2.imwrite('ROI_{}.png'.format(contour_number), ROI)
        contour_number += 1
    print("canny")
    cv2_imshow(canny)
    print('image')
    cv2_imshow(image)

#using Img2table to detext text regions from the image.
def findTextRegion2(image):
      image1 = im.fromarray(image);
      image1.save('image.png')
      doc = Image('image.png', detect_rotation=False)
      assert doc is not None, "file could not be read, check with os.path.exists()"
      extracted_tables = doc.extract_tables(
                                    implicit_rows=False,
                                    borderless_tables=False,
                                    min_confidence=98)
      mark_list = []
      for table in extracted_tables:
        for id_row, row in enumerate(table.content.values()):
          for id_col, cell in enumerate(row):
            # if id_row==3 and id_col>0:
              x1 = cell.bbox.x1+5
              y1 = cell.bbox.y1+5
              x2 = cell.bbox.x2-5
              y2 = cell.bbox.y2-5
              value = cell.value
              #print(type(value))
              # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
              cropped_cell = image[y1:y2, x1:x2]
              if cropped_cell.shape[0] > 0 and cropped_cell.shape[1] > 0:
                print("Row ID:"+str(id_row)+ " COl ID:"+str(id_col))
                # cv2_imshow(cropped_cell)
                # cv2.putText(image, str(id_row) + str(id_col), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (36,255,12), 2)
                hsv = cv2.cvtColor(cropped_cell, cv2.COLOR_BGR2HSV)

                # Define the lower and upper boundaries of the red color in the HSV color space
                lower_red = np.array([0, 50, 50])
                upper_red = np.array([10, 255, 255])

                # Convert the image to binary
                thresh = cv2.threshold(cropped_cell, 127, 255, cv2.THRESH_BINARY_INV)[1]

                # Display the image
                print('Image')
                cv2_imshow(thresh)

                #TODO
                gray_mark = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
                assert img is not None, "file could not be read, check with os.path.exists()"
                edges = cv2.Canny(gray_mark,100,250)
                #edges = cv2.bitwise_not(edges)
                cv2_imshow(edges)

                mark_list.append(edges)

                if thresh.size != 0:  # Check if thresh is not empty
                    #Detecting the tables that contains the mark data by using pytesseract ocr by detecting the keyword "Question"
                    options = "--psm 10"
                    results = pytesseract.image_to_string(
                        cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB),
                        config=options
                    )
                    print('Result:')
                    print(results)

                    results = pytesseract.image_to_string(
                        cv2.cvtColor(gray_mark, cv2.COLOR_GRAY2RGB),
                        config=options
                    )
                    print('Result:')
                    print(results)


              else:
                print("empty cell!")





In [ ]:
for table in markTables:
    findTextRegion2(table)